In [ ]:
---
title: First-order PDE, a taste of shocks (2D)
description: Method of characteristics
author: Daning H.
show-code: False
show-prompt: False
---

We consider a first-order PDE
$$
\begin{aligned}
&\, u_t + uu_x = 0 \\
&\, u(x,0) = 1-x^2
\end{aligned}
$$

We show three animated plots:
1. Moving frame: Behavior of the solution as we move along the characteristics.
2. Fixed frame: Behavior of the solution if we stay at the initial coordinates.
3. The complete solution in $(x,t)$ plane, with a few characteristics and the moving frame visualized.

Think about: What happens in the white region when $x>1$?  The characteristics seem to merge at $x=1$, $t=1/2$; what happens after they merge?

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

Nr = 51

def genSrf(x, t, fchr, fode):
    _X, _T = np.meshgrid(x, t)
    _R = fchr(_X, _T)
    _F = fode(_R, _X, _T)
    return _X, _T, _F

def genChr(r, t, finv, fode):
    _R, _T = np.meshgrid(r, t)
    _X = finv(_R, _T)
    _F = fode(_R, _X, _T)
    return _X, _T, _F

# X1, T1, F1 represent the solution to PDE on the x-t plane.
# X2, T2, F2 represent the propagation of initial solution on the x-t plane.

xf = 1.5
tf = 0.5
rf = 1.0
x = np.linspace(-xf, xf, 121)
t = np.linspace(0.0, tf, Nr)
r = np.linspace(-rf, rf, Nr)

fini = lambda r: 1-r**2

case = '1'
if case == '1':
    def _det(x, t):
        _d = 1+4*t**2-4*x*t
        _m = _d<0
        _d[_m] = np.nan
        _d[~_m] = np.sqrt(_d[~_m])
        return _d
    tt = r"$u_t+uu_x=0$"
    zl = [0,3]
    fc = lambda x, t: (1-_det(x,t)) / (2*t+1e-6)
    fi = lambda x0, t: (1-x0**2)*t+x0
    fo = lambda r, x, t: (-1+2*t*x+_det(x,t)) / (2*t**2+1e-6)

X1, T1, F1 = genSrf(x, t, fc, fo)
X2, T2, F2 = genChr(r, t, fi, fo)
F0 = np.zeros((Nr,))

fig = plt.figure(figsize=(8,10))

a1 = fig.add_subplot(4,1,1)  # Solution over eta
l1, = a1.plot([], [], 'b-', label=r'Moving Frame, $u(\eta,t)$')
a1.plot(X1[0], F1[0], 'k:', label='Initial condition')
a1.legend(loc=2)
a1.set_ylabel('f')
a1.set_title(tt)

a2 = fig.add_subplot(4,1,2)  # Solution over x
l2, = a2.plot([], [], 'r-', label=r'Fixed Frame, $u(x,t)$')
v21, = a2.plot([], [], 'g--')
v22, = a2.plot([], [], 'g--')
a2.plot(X1[0], F1[0], 'k:', label='Initial condition')
a2.legend(loc=2)
a2.set_ylabel('f')

a3 = fig.add_subplot(2,1,2)  # Contour plot
a3.contourf(X1, T1, F1)
l3, = a3.plot([], [], 'r-')  # Moving frame
v31, = a3.plot([], [], 'g--')
v32, = a3.plot([], [], 'g--')
a3.set_title(r'$x-t$ Frame')
a3.set_xlabel('x')
a3.set_ylabel('t')

for _i in range(0, Nr, 3):
    a3.plot(X2[:,_i], T2[:,_i], 'k--')

l0, = a3.plot([],[])

for _a in [a1, a2]:
    _a.set_xlim([-xf, xf])
    _a.set_ylim([-0.05, 1.1])

def animate(i):
    l1.set_data(r, F2[i])
    l2.set_data(X2[i], F2[i])
    v21.set_data([X2[i,0], X2[i,0]], [-0.05, 1.1])
    v22.set_data([X2[i,-1], X2[i,-1]], [-0.05, 1.1])
    l3.set_data(X2[i], T2[i])
    v31.set_data([X2[i,0], X2[i,0]], [0, tf])
    v32.set_data([X2[i,-1], X2[i,-1]], [0, tf])
    return l0,

def init():
    return animate(0)

ani = animation.FuncAnimation(fig, animate, Nr, interval=25, blit=True, init_func=init)
plt.close(ani._fig)
HTML(ani.to_html5_video())